In [ ]:
import feedback_forensics as ff
import feedback_forensics.app.plotting.paper as paper_plot
import pathlib
from IPython.display import display, Latex

data_path = pathlib.Path("../forensics-data/feedback-forensics-public-results")
fig_save_path = pathlib.Path("./output/png")
tex_save_path = pathlib.Path("./output/tex")

# ensure save path exists
fig_save_path.mkdir(parents=True, exist_ok=True)
tex_save_path.mkdir(parents=True, exist_ok=True)


# save general latex preamble
with open(tex_save_path / "000_preamble.tex", "w") as f:
    f.write(paper_plot.get_latex_doc_preamble())

# example latex table
with open(tex_save_path / "999_example_table.tex", "w") as f:
    latex = []
    latex = paper_plot.add_table_preamble(latex, "Example Table")
    latex = paper_plot.add_table_postamble(latex)
    f.write("\n".join(latex))

datasets = [
    ["multipref_10k_v3.json", "MultiPref"],
    ["llama4_arena_vs_public_version.json", "Llama 4 Arena vs Public"],
    ["arena", "Chatbot Arena"],
    ["prism", "PRISM"],
]

cache = {}
for dataset_path, dataset_name in datasets:
    dataset = ff.DatasetHandler(cache=cache)
    dataset.add_data_from_path(data_path / dataset_path)

    overall_metrics = dataset.get_overall_metrics()
    annotator_metrics = dataset.get_annotator_metrics()

    metric_name = "strength"
    strength_metrics = annotator_metrics[dataset_path.split(".")[0]]["metrics"][metric_name]

    kwargs = {}
    if dataset_path == "llama4_arena_vs_public_version.json":
        kwargs = {
            "top_title": "Traits stronger in arena relative to public model",
            "bottom_title": "Traits weaker in arena relative to public model",
        }

    latex_table = paper_plot.get_latex_top_and_bottom_annotators(
        annotator_metrics=strength_metrics,
        metric_name=metric_name.capitalize(),
        **kwargs,
    )

    with open(tex_save_path / f"001_top_and_bottom_annotators_{dataset_path.split('.')[0]}.tex", "w") as f:
        f.write(latex_table)



In [ ]:
# Analysis of Arena data
import pandas as pd

dataset_name = "arena"
dataset = ff.DatasetHandler(cache=cache)
dataset.add_data_from_path(data_path / dataset_name)

general_df = dataset.first_handler.df
values = [
    'Creative Writing Prompts',
    'Songwriting Prompts',
    'Resume and Cover Letter Writing',
    'Professional Email Communication',
]
dataset.split_by_col(col="narrower_category", selected_vals=values)

metrics_df = dataset.get_annotator_metrics_df(metric_name="strength", index_col_name="Generate a response that...")

latex_str = paper_plot.get_latex_table_from_metrics_df(
    metrics_df=metrics_df.head(10),
    title="Encouraged personality traits across writing domains in Chatbot Arena (Strength)",
)

with open(tex_save_path / "002_writing_tasks_arena.tex", "w", encoding="utf-8") as f:
    f.write(latex_str)

In [ ]:
# Analysis of MultiPref data
import pandas as pd
import feedback_forensics as ff
import pathlib

cache = {}
data_path = pathlib.Path("../forensics-data/feedback-forensics-public-results")

dataset_name = "multipref_10k_v3.json"
dataset = ff.DatasetHandler(cache=cache)
dataset.add_data_from_path(data_path / dataset_name)

In [ ]:
annotator_metadata = dataset.get_available_annotators()
special_annotators = {
    annotator_key: metadata
    for annotator_key, metadata in annotator_metadata.items()
    if metadata["variant"] in ["unknown", "human"]
    if "normal" in metadata["annotator_visible_name"] or "expert" in metadata["annotator_visible_name"] or "gpt4" in metadata["annotator_visible_name"]
}
special_annotators

dataset.set_annotator_cols(annotator_keys=list(special_annotators.keys()))
df = dataset.get_annotator_metrics_df(metric_name="strength", index_col_name="Generate a response that...")


In [19]:
rename_dict = {
    'multipref_10k_v3\n(unknown: expert_1_preferred_text)': 'Human Expert 2',
    'multipref_10k_v3\n(unknown: preferred_text_gpt4)': 'GPT-4',
    'multipref_10k_v3\n(unknown: normal_0_preferred_text)': 'Human Regular 1',
    'multipref_10k_v3\n(unknown: normal_1_preferred_text)': 'Human Regular 2',
    'multipref_10k_v3\n(unknown: expert_0_preferred_text)': 'Human Expert 1',
}
# rename the columns
df.rename(rename_dict, inplace=True, axis=1)

# reorder the columns (experts, regular, gpt-4)
df = df[['Generate a response that...', 'Human Expert 1', 'Human Expert 2', 'Human Regular 1', 'Human Regular 2', 'GPT-4', 'Max diff']]

latex_str = paper_plot.get_latex_table_from_metrics_df(
    metrics_df=df.head(5),
    title="Personality traits encouraged by different annotators on MultiPref (Strength)",
    first_col_width=0.15,
)

with open(tex_save_path / "003_cross_annotator_comparison_multipref.tex", "w", encoding="utf-8") as f:
    f.write(latex_str)



In [ ]:
# plotting model analysis
import pandas as pd
import feedback_forensics as ff
import pathlib

results_path = pathlib.Path("exp/outputs/2025-05-12_14-46-21_mc_v1/results/070_annotations_train_ap.json")

dataset = ff.DatasetHandler(cache=cache)
dataset.add_data_from_path(results_path)

annotators = dataset.get_available_annotators()
model_anns = {
    k: v for k, v in annotators.items() if v["variant"] == "model_identity"
}

dataset.set_annotator_cols(annotator_keys=list(model_anns.keys()))
metrics_df = dataset.get_annotator_metrics_df(metric_name="strength", index_col_name="Generate a response that...")

# remove the 070_annotations_train_ap from each column name
metrics_df.columns = metrics_df.columns.str.replace("070_annotations_train_ap\n(Model: ", "").str.replace(")", "").str.replace("openrouter/", "").str.replace(" ", "-")

# rename max-diff to Max diff
metrics_df.rename(columns={"Max-diff": "Max diff"}, inplace=True)

# set all gpt-4o models to 0
metrics_df["openai/gpt-4o-2024-08-06"] = 0

latex_str = paper_plot.get_latex_table_from_metrics_df(
    metrics_df=metrics_df.head(5),
    title="Most diverging personality traits across models",
    first_col_width=0.15,
)

with open(tex_save_path / "004_model_comparison.tex", "w", encoding="utf-8") as f:
    f.write(latex_str)

In [ ]:
list(metrics_df.head(5).columns)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

short_names = {
    'meta-llama/llama-3-70b-instruct': "Llama-3-70b",
    'meta-llama/llama-3.3-70b-instruct':  "Llama-3.3-70b",
    'meta-llama/llama-4-maverick': "Llama-4-Maverick",
    'mistralai/mistral-medium': "Mistral-Medium",
    'mistralai/mistral-medium-3': "Mistral-Medium-3",
    'openai/gpt-4.1-2025-04-14': "GPT-4.1",
    'openai/gpt-4o-2024-08-06': "GPT-4o",
    'meta-llama/llama-2-70b-chat': "Llama-2-70b",
    'openai/gpt-3.5-turbo': "GPT-3.5-Turbo",
    'mistralai/mistral-7b-instruct-v0.1': "Mistral-7b",
}


def plot_model_comparison_by_family(metrics_df, trait_index="makes more confident statements"):
    # Import adjustText for automatic label positioning
    from adjustText import adjust_text

    # Extract data for the plot
    data = metrics_df.loc[trait_index].drop("Max diff")
    # Group models by family
    model_families = {
        "Meta": sorted([col for col in data.index if "llama" in col.lower()]),
        "Mistral": sorted([col for col in data.index if "mistral" in col.lower()]),
        "OpenAI": sorted([col for col in data.index if "gpt" in col.lower()],
                  key=lambda x: 0 if "gpt-4o" in x.lower() else 1 if "gpt-4.1" in x.lower() else -1)
    }

    # Create the plot
    fig = plt.figure(figsize=(3.5, 2))

    # Plot each model family with a different color
    colors = ['#1f77b4', '#ff7f0e', '#2ca02c']  # Blue, Orange, Green
    markers = ['o', 's', '^']

    # Store text objects for adjustText and lines for objects to avoid
    texts = []
    line_objects = []
    x_coords = []
    y_coords = []

    for i, (family, models) in enumerate(model_families.items()):
        family_data = data[models]
        # Normalize x positions to ensure all model families start at 0 and end at 1
        if len(models) > 1:
            x = np.linspace(0, 1, len(models)) + 1.2 * i
        else:
            x = np.array([0.5])  # Center a single model
        line, = plt.plot(x, family_data.values, marker=markers[i], linestyle='-',
                 label=family, color=colors[i], linewidth=2, markersize=8)
        line_objects.append(line)

        # Add model names as x-tick labels and collect text objects
        for j, model in enumerate(models):
            label = short_names[model]
            text_obj = plt.text(x[j], family_data.values[j],
                     label,
                     ha='center', va='center', fontsize=8)
            texts.append(text_obj)

            x_coords.append(x[j])
            y_coords.append(family_data.values[j])

    plt.axhline(y=0, color='gray', linestyle='--', alpha=0.7)
    plt.grid(True, linestyle=':', alpha=0.7)
    plt.ylabel(f'Strength of trait')
    plt.xticks([])
    plt.yticks(plt.yticks()[0], minor=True)
    plt.tick_params(axis='y', which='minor', left=False)
    plt.title(f'{trait_index.capitalize()}')
    plt.legend()

    # Use adjustText to automatically position labels without overlap
    # Add line_objects to objects parameter to make labels avoid the plotted lines
    adjust_text(texts,
                objects=line_objects,
                arrowprops=dict(arrowstyle='-', color='gray', lw=0.5),
                #x = x_coords,
                #y = y_coords,
                #expand_points=(1.5, 1.5),
                #expand=(1.1, 1.1),
                #force_text=(0.5, 0.8),
                #force_objects=(0.5, 0.8)
    )

    plt.tight_layout()
    return fig

# Example usage - plot for confidence
traits = [
    "makes more confident statements",
    "provides a numbered list format",
    "has a friendlier tone",
    "ends with a follow-up question",
    "expresses more emotion",
]

comparison_path = fig_save_path / "model_comparison"
comparison_path.mkdir(parents=True, exist_ok=True)

for trait in traits:
    fig = plot_model_comparison_by_family(metrics_df, trait)
    fig.savefig(comparison_path / (trait.replace(" ", "_") + ".png"), dpi=300)